<a href="https://colab.research.google.com/github/vincentmartin/tp-initiation-llm-student-version/blob/main/TP_initiation_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP d'initiation aux LLM

Dans ce TP, vous allez apprendre les bases de l'IA générative en manipulant et en contrôlant un LLM installé en local.

En sortie de ce module, vous serez capable de :
- Installer et importer les dépendances nécessaires
- Interroger un LLM pour répondre à tout type de question, comme avec chatGPT
- Analyser le fonctionnement d'un LLM
- Utiliser un LLM pour résumer une conversation
- Explorer les techniques de zero-shot, one-shot et few-shot inference

### Instruction à suivre pour exécution sur Google Colab

Aller dans `Execution -> Modifier le type d'exécution` puis sélectionner `T4-GPU` pour exploiter les fonctionnalités GPU.

![Colab GPU](resources/colab_gpu.png "T4-GPU")

## Installation des dépendances

Installons les dépendances nécessaires :
- **transformers** : la bibliothèque permettant de mettre en oeuvre les LLM exploitant le modèle transformers
- **torch** : célèbre bibliothèque de deep learning, sous jacente à transformers

In [1]:
%pip install -U datasets

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch \
    torchdata

%pip install \
    transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstallin

Chargeons les dépendances.

**Remarque : Si l'exécution ressort en erreur ; tenter de recharger les dépendances.**

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

## Chargement du LLM

Pour interagir avec un LLM, nous allons d'abord devoir le télécharger. Pour cet exemple, nous choissons un modèle simple et "léger" : flan-t5.

Nous chargeons également le **Tokenizer** afin de convertir le texte en tokens et vice-versa.

In [3]:
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

**Exercice** : en vous aidant de la documentation https://huggingface.co/docs/transformers/llm_tutorial :
- Générer et afficher les tokens (ids) de la phrase (encodage)
- Décoder la liste de tokens (ids) et afficher la phrase (décodage)

In [6]:
# Charger le modèle et le tokenizer
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Phrase à encoder et décoder
sentence = "Que peux-tu me dire sur les LLMs ?"

# Encodage de la phrase en tokens (IDs)
encoded = tokenizer(sentence, return_tensors="pt")
token_ids = encoded['input_ids'][0].tolist()

print("Tokens (IDs) :", token_ids)

# Décodage de la liste de tokens en phrase
decoded_sentence = tokenizer.decode(token_ids, skip_special_tokens=True)

# Afficher la phrase décodée
print("Phrase décodée :", decoded_sentence)


Tokens (IDs) : [7227, 3, 16162, 18, 17, 76, 140, 5794, 244, 110, 301, 11160, 7, 3, 58, 1]
Phrase décodée : Que peux-tu me dire sur les LLMs ?


## Interrogation du LLM

A présent, utilisons notre LLM pour générer du texte.

Notez la syntaxe `User: question? Assistant: "`. Nous utilisons cette syntaxe car le LLM est un modèle qui génère la suite de la phrase et cette syntaxe lui permet de comprendre ce qu'on attend de lui. Ceci à la différence des modèles d'instruction qui génèrent une réponse pour une instruction donnée.

In [7]:
sentence = "User: quelle est la capitale de la france ?Assistant: "

inputs = tokenizer(sentence, return_tensors='pt') # return les tenseurs au format pytorch
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50
    )[0],
    skip_special_tokens=True # Ne pas retourner les tokens <s>, </s>, ...
)

print(output)

Paris


C'est assez basique pour l'instant mais ne vous inquiétez pas, ce n'est que le premier TP ;).

## Résumé de dialogues

Dans cette partie, nous allons utiliser le LLM pour résumer des dialogues.

Tout d'abord, téléchargeons le dataset [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) depuis Huggingface

In [8]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Affichons 2 exemples de dialogues, les exemples numéro 40 et 200.

In [9]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('DIALOGUE D ENTREE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('RESUME HUMAIN:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
DIALOGUE D ENTREE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
RESUME HUMAIN:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example  

Tentons une première approche pour résumer les dialogues 40 et 200.

In [10]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt') # retourner les tenseurs
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50, # max 50 tokens générés
        )[0],
        skip_special_tokens=True # on ne génère pas les tokens spéciaux <, >, ...
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'DIALOGUE D ENTREE::\n{dialogue}')
    print(dash_line)
    print(f'RESUME HUMAIN:\n{summary}')
    print(dash_line)
    print(f'RESUME PAR LLM SANS PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
DIALOGUE D ENTREE::
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
RESUME HUMAIN:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
RESUME PAR LLM SANS PROMPT ENGINEERING:
Person1: It's ten to nine.

-----------------------------------------

**Exercice** : selon vous est-ce que le résumé est bon ? Pourquoi ?

Le résumé généré par le LLM n'est pas bon car il ne reflète ni le contenu ni l'intention des dialogues.

## Résumé avec un prompt Instruction

Dans l'exemple ci-dessous, ajoutons une instruction indiquant au LLM ce qu'il doit faire.

### 1. Zero shot inference

Pour amener le modèle à accomplir une tâche, comme résumer un dialogue, vous pouvez transformer ce dialogue en une consigne spécifique. Cela est connu sous le nom d'inférence zéro-shot.

En encadrant le dialogue dans une consigne descriptive, vous pourrez observer les modifications apportées au texte généré.

In [11]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Summarize the following conversation between two persons to extract the key points of the conversation.

{dialogue}

Summary:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'DIALOGUE D ENTREE:\n{prompt}')
    print(dash_line)
    print(f'RESME HUMAIN:\n{summary}')
    print(dash_line)
    print(f'>>>RESME AVEC ZERO SHOT INFERENCE:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
DIALOGUE D ENTREE:

Summarize the following conversation between two persons to extract the key points of the conversation.

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary:
    
---------------------------------------------------------------------------------------------------
RESME HUMAIN:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
------------------------------------------------------------------------------------------

C'est déjà mieux ! Mais on peut encore faire mieux. Essayons de rajouter un exemple de résumé.

### 2. One Shot Inference

L'inférence one-shot et few-shot consiste à fournir au modèle de langage un ou plusieurs exemples complets de paires consigne-réponse correspondant à votre tâche avant de lui soumettre la consigne réelle que vous souhaitez qu'il accomplisse. Cela s'appelle "l'apprentissage en contexte" (_in context learning_), et cela permet au modèle de comprendre votre tâche spécifique. Pour en savoir plus, vous pouvez consulter [cet article](https://huggingface.co/blog/few-shot-learning-gpt-neo-and-inference-api).

Définissons une fonction qui permet de générer un prompt avec 1 exemple de dialogue et son résumé.

In [12]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

Summary:
{summary}


"""

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f"""
Dialogue:

{dialogue}

Summary:
"""

    return prompt

Construsons le prompt et affichons le.

In [13]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)
# one_shot_prompt is a string
print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more 

Lançons l'inférence sur un dialogue, qui doit bien entendu être différent de celui utilisé pour réaliser l'exemple.

In [14]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'RESME HUMAIN:\n{summary}\n')
print(dash_line)
print(f'>>>RESME LLM AVEC ONE SHOT INFERENCE:\n{output}')

---------------------------------------------------------------------------------------------------
RESME HUMAIN:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
>>>RESME LLM AVEC ONE SHOT INFERENCE:
#Person1#: You could add a painting program to your software. #Person2#: That would be a bonus. #Person1#: You might also want to upgrade your hardware because it is pretty outdated now


Voilà qui est encore mieux !!

### 3. Few shot inference

Essayons à présent de fournir 3 exemples de paires (dialogue, résumé). C'est ce que l'on appelle le **few shot inference**.

In [15]:
example_indices_full = [84, 85, 86] # exemples à fournir
example_index_to_summarize = 201 # dialogue à résumer

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: Stupid girl, making me spend so much money, now I have to get it from the ATM...
#Person2#: Hello, welcome to Universal Bank. Please insert your card into the slot.
#Person1#: I know where to put my card! Stupid machine, talking to me like I ' m an idiot...
#Person2#: Please input your 6 digit PIN code followed by the pound key. Thank you. Please select an option. Thank you. You have selected withdrawal.
#Person1#: Yeah, yeah, I know what I selected. Just gimme my money!
#Person2#: Please type the amount you would like to withdraw. Thank you, you want to transfer 10000 USD to the World Wildlife Foundation. If this is correct please press 1.
#Person1#: No, no! Stupid machine, what are you doing! No!
#Person2#: Confirmed. Thank you for using our bank! Please remove your card from the slot. Goodbye!
#Person1#: No, no way! What happened? Give me my money!
#Person2#: Danger, danger! The exits have been sealed and the doors will remain locked in until the local authori

In [16]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'RESUME HUMAIN:\n{summary}\n')
print(dash_line)
print(f'>>>RESUME LLM AVEC FEW SHOT INFERENCE:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (1384 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
RESUME HUMAIN:
#Person1# is driving #Person2# to an inn. They talk about their careers, ages, and where they was born.

---------------------------------------------------------------------------------------------------
>>>RESUME LLM AVEC FEW SHOT INFERENCE:
#Person1 is flying to China from Mexico. #Person2 is from Mexico. #Person1 is from Mexico. #Person2 is from Mexico. #Person1 is from China. #Person2 is from Mexico


**Exercice** : modifier les exemples fournis en entrée et indiquer ce que vous contacter en commentaire dans une section markdown.

## Influence des paramètres du LLM

Nous allons maintenant faire varier plusieurs paramètres du LLM :
- température
- top_k
- top_p
- sampling

Pour cela aidez-vous de la documentation :
- https://huggingface.co/docs/transformers/generation_strategies
- https://huggingface.co/docs/transformers/main_classes/text_generation

**Exercice** : créer une fonction qui prend les 4 paramètres ci-dessous et le paramètres _few_shot_prompt_ défini précédemment et qui retourne le résultat de la génération.

In [18]:
# Charger le modèle et le tokenizer
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Fonction pour générer un résumé en jouant sur les paramètres de génération
def generate_summary(temperature, top_k, top_p, sampling, prompt):
    generation_args = {
        "temperature": temperature,
        "top_k": top_k,
        "top_p": top_p,
        "do_sample": sampling,
        "max_length": 100,
    }

    # Encodage du prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Génération du texte
    outputs = model.generate(**inputs, **generation_args)

    # Décodage et retour du résultat
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Exemple d'utilisation
prompt = "#Person1#: What time is it, Tom?\n#Person2#: Just a minute. It's ten to nine by my watch.\n#Person1#: Is it? I had no idea it was so late. I must be off now.\n#Person2#: What's the hurry?\n#Person1#: I must catch the nine-thirty train.\n#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there."
result = generate_summary(temperature=0.7, top_k=50, top_p=0.9, sampling=True, prompt=prompt)
print("Résumé généré :", result)

Résumé généré : Person1: I'm on the way.


**Exercice** : expliquer en 1 ou 2 lignes l'influence de chacun des 4 paramètres (dans une section markdown).

# Explication des paramètres de génération
# - **Température** : Contrôle la créativité du modèle. Une valeur plus élevée génère des réponses plus diversifiées mais moins déterministes.
# - **Top-k** : Limite le choix des prochains mots aux k mots les plus probables, réduisant le risque de sorties improbables.
# - **Top-p** : Utilise le "nucleus sampling" en limitant la sélection aux mots dont la probabilité cumulée atteint p, favorisant un équilibre entre diversité et cohérence.
# - **Sampling** : Active ou désactive le tirage aléatoire pour la génération. Si désactivé, le modèle choisit toujours le mot le plus probable.
